## Pydantic fundamentals

In [4]:
from pydantic_ai import Agent

agent = Agent(model="google-gla:gemini-2.5-flash")
# await betyder att vi väntar på ett asynkront resultat (agenten ska bli klar)
result = await agent.run("Give me an IT employee working in Sweden, no yapping")

result

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


AgentRunResult(output='**Name:** Linnea Andersson\n**Role:** Senior Cybersecurity Analyst\n**Company:** Securify Solutions AB\n**Location:** Gothenburg, Sweden\n**Skills:** SIEM (Splunk, Elastic), Threat Detection, Incident Response, Penetration Testing, ISO 27001, Python scripting.\n**Focus:** Protecting organizational data and infrastructure, identifying vulnerabilities, and developing robust security protocols.')

In [5]:
print(result.output)

**Name:** Linnea Andersson
**Role:** Senior Cybersecurity Analyst
**Company:** Securify Solutions AB
**Location:** Gothenburg, Sweden
**Skills:** SIEM (Splunk, Elastic), Threat Detection, Incident Response, Penetration Testing, ISO 27001, Python scripting.
**Focus:** Protecting organizational data and infrastructure, identifying vulnerabilities, and developing robust security protocols.


### Use pydantic to structure output

In [7]:
from pydantic import BaseModel, Field
# description är en typ av prompt för LLM:en, använd det alltid
class Employee(BaseModel):
    name: str = Field(description="first name and last name of the employee")
    # gt = greater than, lt = less than
    age: int = Field(gt = 17, lt = 67, description="age of the employee in years")
    salary: int = Field(gt = 30_000, lt = 70_000, description="monthly salary of the employee in SEK, the more senior the employee is, the higher the salary")
    role: str = Field(description="the role of the employee in the company. Always include the seniority in the role (junior, mid-level, senior, lead etc.)")
    desciption: str = Field(description="a short description (max 2 sentences) of the employee's skills and experience, include keywords relevant to the role")

result = await agent.run("Give me an IT employee working as data engineer in Sweden.", output_type=Employee)

print(result)
    
    

AgentRunResult(output=Employee(name='Erik Andersson', age=32, salary=50000, role='Mid-level Data Engineer', desciption='Erik specializes in building and maintaining robust data pipelines. He has strong skills in Python, SQL, and cloud platforms like AWS.'))


In [8]:
employee = result.output
employee

Employee(name='Erik Andersson', age=32, salary=50000, role='Mid-level Data Engineer', desciption='Erik specializes in building and maintaining robust data pipelines. He has strong skills in Python, SQL, and cloud platforms like AWS.')

In [9]:
employee.name, employee.desciption

('Erik Andersson',
 'Erik specializes in building and maintaining robust data pipelines. He has strong skills in Python, SQL, and cloud platforms like AWS.')

In [10]:
isinstance(employee, BaseModel)

True

In [11]:
employee.model_dump()

{'name': 'Erik Andersson',
 'age': 32,
 'salary': 50000,
 'role': 'Mid-level Data Engineer',
 'desciption': 'Erik specializes in building and maintaining robust data pipelines. He has strong skills in Python, SQL, and cloud platforms like AWS.'}

In [12]:
employee.model_dump_json()

'{"name":"Erik Andersson","age":32,"salary":50000,"role":"Mid-level Data Engineer","desciption":"Erik specializes in building and maintaining robust data pipelines. He has strong skills in Python, SQL, and cloud platforms like AWS."}'

In [13]:
print(employee.model_dump_json(indent=4))

{
    "name": "Erik Andersson",
    "age": 32,
    "salary": 50000,
    "role": "Mid-level Data Engineer",
    "desciption": "Erik specializes in building and maintaining robust data pipelines. He has strong skills in Python, SQL, and cloud platforms like AWS."
}


#### Simulate 10 employees

In [15]:
result = await agent.run("Simulate 10 employees in AI, data science, data engineering, data analyst field", output_type=list[Employee])

result


AgentRunResult(output=[Employee(name='Alice Smith', age=32, salary=65000, role='Senior AI Engineer', desciption='Specialized in machine learning model development and deployment. Expertise in Python, TensorFlow, and cloud platforms.'), Employee(name='Bob Johnson', age=25, salary=40000, role='Junior AI Researcher', desciption='Focuses on exploring novel AI algorithms and data analysis. Proficient in PyTorch and research methodologies.'), Employee(name='Charlie Brown', age=38, salary=68000, role='Lead Data Scientist', desciption='Leads data science projects from conception to implementation. Strong background in predictive modeling, A/B testing, and team leadership.'), Employee(name='Diana Miller', age=29, salary=55000, role='Mid-level Data Scientist', desciption='Develops and implements statistical models for business insights. Experienced in R, SQL, and data visualization tools.'), Employee(name='Eve Davis', age=24, salary=38000, role='Junior Data Scientist', desciption='Assists in dat

In [16]:
for employee in result.output:
    print(f"{employee.name} is {employee.age} years old, and earns {employee.salary} SEK per month as a {employee.role}. Description: {employee.desciption}")

Alice Smith is 32 years old, and earns 65000 SEK per month as a Senior AI Engineer. Description: Specialized in machine learning model development and deployment. Expertise in Python, TensorFlow, and cloud platforms.
Bob Johnson is 25 years old, and earns 40000 SEK per month as a Junior AI Researcher. Description: Focuses on exploring novel AI algorithms and data analysis. Proficient in PyTorch and research methodologies.
Charlie Brown is 38 years old, and earns 68000 SEK per month as a Lead Data Scientist. Description: Leads data science projects from conception to implementation. Strong background in predictive modeling, A/B testing, and team leadership.
Diana Miller is 29 years old, and earns 55000 SEK per month as a Mid-level Data Scientist. Description: Develops and implements statistical models for business insights. Experienced in R, SQL, and data visualization tools.
Eve Davis is 24 years old, and earns 38000 SEK per month as a Junior Data Scientist. Description: Assists in dat

### More complex models

In [20]:
from typing import Literal


class Department(BaseModel):
    name: Literal["IT", "HR", "Marketing"]= Field(description="name of the department")
    description: str = Field(description="a short description of the department's responsibilities")
    employees: list[Employee]
    
class Company(BaseModel):
    name: str = Field(description="name of the company")
    departments: list[Department]
    description: str = Field(description="a short description (two sentences) of the company")
    
result = await agent.run(("Give me 2-3 employees in each department", "Company should be called CoolCompany and sell ice creams"), output_type=Company)

In [22]:
company = result.output
company.__dict__.keys()

dict_keys(['name', 'departments', 'description'])

In [23]:
company.name

'CoolCompany'

In [26]:
[department.name for department in company.departments]

['IT', 'HR', 'Marketing']

In [ ]:
departments = [department.name for department in company.departments]
departments[0].employees

In [28]:
company.model_dump()

{'name': 'CoolCompany',
 'departments': [{'name': 'IT',
   'description': "The IT department is responsible for managing CoolCompany's technology infrastructure, developing software solutions, and ensuring data security.",
   'employees': [{'name': 'Alice Wonderland',
     'age': 35,
     'salary': 55000,
     'role': 'Senior Software Engineer',
     'desciption': 'Alice is an experienced software engineer with a strong background in full-stack development. She specializes in creating scalable web applications and optimizing system performance.'},
    {'name': 'Bob Thebuilder',
     'age': 28,
     'salary': 40000,
     'role': 'Mid-level IT Support Specialist',
     'desciption': 'Bob provides essential technical support, troubleshooting hardware and software issues. He is proficient in network administration and user assistance.'}]},
  {'name': 'HR',
   'description': 'The HR department handles all aspects of human resources, including recruitment, employee relations, and professiona